<a href="https://colab.research.google.com/github/MarkKMensah/ASR-app/blob/main/Testv3_turbo_augument.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [27]:
import zipfile
import os

# Adjust path if stored in a subfolder
zip_path = "/content/drive/MyDrive/audios.zip"
extract_dir = "/content/asr_test_data"

# Unzip
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print("Extracted files:")
print(os.listdir(extract_dir))


Extracted files:
['ak_gh_image_0803_u123_1_1688939334131_00974.mp3', 'ak_gh_image_0072_u133_1_1688920839618_12823.mp3', 'ak_gh_image_0024_u133_1_1688938029258_03229.mp3', 'ak_gh_image_0018_u119_1_1688933121338_15873.mp3', 'ak_gh_image_0093_u133_1_1688921086009_06529.mp3', 'audios']


In [16]:
import pandas as pd
import os

# Path to the audios folder
audio_dir = "/content/asr_test_data/audios"
metadata_path = os.path.join(audio_dir, "metadata.xlsx")

# Load metadata
metadata_df = pd.read_excel(metadata_path)

# Normalize file names from the AUDIO_PATH field
metadata_df['file_name'] = metadata_df['AUDIO_PATH'].apply(lambda x: os.path.basename(str(x)).strip())
metadata_df['Transcriptions'] = metadata_df['Transcriptions'].astype(str).str.strip()

# Show first few rows
metadata_df[['file_name', 'Transcriptions']].head()


,file_name,Transcriptions
0,Akan\selected transcribed audios\audios\ak_gh_...,"Adwumayɛfoɔ nsia, adwumayɛfoɔ nson a wɔreyɛ ad..."
1,Akan\selected transcribed audios\audios\ak_gh_...,"Baabi a yɛbu fangoo, na ɛhɔ ahye ama nwisie ɛf..."
2,Akan\selected transcribed audios\audios\ak_gh_...,Atoyerɛnkyɛm asi wɔ bea a wɔgu fangoo ɛgu ɛhyɛ...
3,Akan\selected transcribed audios\audios\ak_gh_...,Mmmaa mmienu ɛne sukuu nkwadaa mmiɛnsa ɛgyina ...
4,Akan\selected transcribed audios\audios\ak_gh_...,Nnipa nnum koto hɔ a nhwiren sisi wɔn anim. Wɔ...


In [20]:
from huggingface_hub import login

login()


In [24]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration
import torchaudio
import torch

# Load the standard processor and your fine-tuned Akan model
processor = WhisperProcessor.from_pretrained("openai/whisper-large-v3")
model = WhisperForConditionalGeneration.from_pretrained(
    "cdli/whisper-large_v3_turbo_Akan_standardspeech_spec_and_audio_augment"
)
model.eval()

# Set device (GPU if available)
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)


WhisperForConditionalGeneration(
  (model): WhisperModel(
    (encoder): WhisperEncoder(
      (conv1): Conv1d(128, 1280, kernel_size=(3,), stride=(1,), padding=(1,))
      (conv2): Conv1d(1280, 1280, kernel_size=(3,), stride=(2,), padding=(1,))
      (embed_positions): Embedding(1500, 1280)
      (layers): ModuleList(
        (0-31): 32 x WhisperEncoderLayer(
          (self_attn): WhisperSdpaAttention(
            (k_proj): Linear(in_features=1280, out_features=1280, bias=False)
            (v_proj): Linear(in_features=1280, out_features=1280, bias=True)
            (q_proj): Linear(in_features=1280, out_features=1280, bias=True)
            (out_proj): Linear(in_features=1280, out_features=1280, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1280, out_features=5120, bias=True)
          (fc2): Linear(in_features=5120, out_features=1280, bia

In [30]:
import os

actual_files = os.listdir("/content/asr_test_data/audios")
print("Actual file names:")
print(actual_files)


Actual file names:
['ak_gh_image_0803_u123_1_1688939334131_00974.mp3', 'ak_gh_image_0072_u133_1_1688920839618_12823.mp3', 'metadata.xlsx', '~$selected transcribed audios (1).xlsx', 'ak_gh_image_0024_u133_1_1688938029258_03229.mp3', 'selected transcribed audios (1).xlsx', 'ak_gh_image_0018_u119_1_1688933121338_15873.mp3', 'ak_gh_image_0093_u133_1_1688921086009_06529.mp3', '~$metadata.xlsx']


In [31]:
print("From metadata:", metadata_df['file_name'].iloc[0])


From metadata: Akan\selected transcribed audios\audios\ak_gh_image_0007_u14_1_1687326579883_00018.mp3


In [36]:
import os

audio_dir = "/content/asr_test_data/audios"

print("🔎 Files in /audios:")
for f in os.listdir(audio_dir):
    print(f)


🔎 Files in /audios:
ak_gh_image_0803_u123_1_1688939334131_00974.mp3
ak_gh_image_0072_u133_1_1688920839618_12823.mp3
metadata.xlsx
~$selected transcribed audios (1).xlsx
ak_gh_image_0024_u133_1_1688938029258_03229.mp3
selected transcribed audios (1).xlsx
ak_gh_image_0018_u119_1_1688933121338_15873.mp3
ak_gh_image_0093_u133_1_1688921086009_06529.mp3
~$metadata.xlsx


In [37]:
# Use an actual file present in the folder
test_file = "ak_gh_image_0072_u133_1_1688920839618_12823.mp3"
file_path = os.path.join(audio_dir, test_file)

print("📂 Full path:", file_path)
print("✅ File exists:", os.path.isfile(file_path))


📂 Full path: /content/asr_test_data/audios/ak_gh_image_0072_u133_1_1688920839618_12823.mp3
✅ File exists: True


In [38]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration
import torch
import librosa
import pandas as pd
import os
from jiwer import wer
from tqdm import tqdm

# Paths
audio_dir = "/content/asr_test_data/audios"
metadata_path = os.path.join(audio_dir, "metadata.xlsx")

# Load metadata and clean file names
metadata_df = pd.read_excel(metadata_path)
metadata_df['file_name'] = metadata_df['AUDIO_PATH'].apply(
    lambda x: os.path.basename(str(x).replace("\\", "/")).strip()
)
metadata_df['Transcriptions'] = metadata_df['Transcriptions'].astype(str).str.strip()

# Load model and processor
processor = WhisperProcessor.from_pretrained("openai/whisper-large-v3")
model = WhisperForConditionalGeneration.from_pretrained("cdli/whisper-large_v3_turbo_Akan_standardspeech_spec_and_audio_augment")
model.eval()
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# Get actual .mp3 files in the folder (limit to 5 for your case)
test_files = [f for f in os.listdir(audio_dir) if f.endswith(".mp3")][:5]

# Store results
results = []

# Loop through and transcribe
for file_name in tqdm(test_files, desc="Processing files"):
    audio_path = os.path.join(audio_dir, file_name)

    # Load audio using librosa (guaranteed to work)
    waveform, sample_rate = librosa.load(audio_path, sr=16000)
    waveform = torch.tensor(waveform).unsqueeze(0)  # Shape (1, samples)

    # Preprocess and run model
    inputs = processor(waveform.squeeze(), sampling_rate=16000, return_tensors="pt").to(device)
    with torch.no_grad():
        predicted_ids = model.generate(**inputs)
    transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)[0].strip()

    # Match ground truth
    ground_row = metadata_df[metadata_df['file_name'] == file_name]
    if ground_row.empty:
        print(f"⚠️ Ground truth not found for {file_name}")
        continue
    ground_truth = ground_row['Transcriptions'].values[0].strip()

    # Compute WER
    error = wer(ground_truth.lower(), transcription.lower())

    # Save result
    results.append({
        "Audio File": file_name,
        "Ground Truth": ground_truth,
        "Model Transcript": transcription,
        "WER": round(error, 3)
    })

# Show results
results_df = pd.DataFrame(results)
from IPython.display import display
display(results_df)

# Save to CSV if needed
results_df.to_csv("/content/asr_evaluation_results.csv", index=False)
print("✅ WER results saved as 'asr_evaluation_results.csv'")


Processing files: 100%|██████████| 5/5 [05:16<00:00, 63.35s/it]


,Audio File,Ground Truth,Model Transcript,WER
0,ak_gh_image_0803_u123_1_1688939334131_00974.mp3,Abranteɛ bi gyina dwam. Na ɔhyɛ ataade soro. N...,"Aberanteɛ bi gyina dwam, na ɔhyɛ atade soro, n...",0.200
1,ak_gh_image_0072_u133_1_1688920839618_12823.mp3,"Nkorɔfoɔ bi gyina po ano, wɔn mu dodoɔ no ara ...",Nkurɔfoɔ bi gyina po ano. Wɔn mu dodoɔ no ara ...,0.243
2,ak_gh_image_0024_u133_1_1688938029258_03229.mp3,Buroni akwadaa bi redi agorɔ. Ɔde label ahyehy...,Bronin akwadaa bi redi agorɔ. Ɔde label ahyehy...,0.061
3,ak_gh_image_0018_u119_1_1688933121338_15873.mp3,"Akwanhyia bi ɛsi wɔ akwantinfi na ɛfiri ɛbien,...","Akwanhyia bi asi wɔ akwantinfi, na afiri abien...",0.355
4,ak_gh_image_0093_u133_1_1688921086009_06529.mp3,Aha yɛ kwan ho. Nhyɛn ahodoɔ redi akɔneaba. Kw...,Aha yɛ kwan ho. Hyɛn ahodoɔ redi akɔnneaba. Kw...,0.243


✅ WER results saved as 'asr_evaluation_results.csv'


In [39]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [41]:
!cp /content/asr_evaluation_results.csv /content/drive/MyDrive/ASR_Evaluations/


cp: cannot create regular file '/content/drive/MyDrive/ASR_Evaluations/': Not a directory
